In [1]:
import os
import sys
import random
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np


from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import model_from_json
from sklearn.metrics import f1_score


Using TensorFlow backend.


In [2]:
seed = np.random.randint(0, 10000)
np.random.seed(seed)


TEST_SIZE = 50
IMG_PATCH_SIZE = 16 # image size should be an integer multiple of this number!
NUM_CHANNELS = 3 # RGB images
FOREGROUND_THRESHOLD = 0.25  # percentage of pixels > 1 required to assign a foreground label to a patch
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
WINDOW_SIZE = 16


In [3]:
def data_augmentation(directory_name): 
    datagen = ImageDataGenerator()
    filenames = os.listdir(directory_name)
    
    #create 24 rotated images for one image
    angls = np.arange(0, 360, 15)
    zooms = np.array([1., 0.85, 0.8, 0.75, 0.8, 0.85,
                      1., 0.85, 0.8, 0.75, 0.8, 0.85,
                      1., 0.85, 0.8, 0.75, 0.8, 0.85,
                      1., 0.85, 0.8, 0.75, 0.8, 0.85])
    imgs = []
    
    for i, fileNb in enumerate(filenames):
        img=mpimg.imread('../Datasets/training/images/'+fileNb)
        imgr = img_to_array(img)
        for j, angle in enumerate(angls):
            zoom = zooms[j]
            img2 = datagen.apply_transform(x=imgr, transform_parameters={'theta':angle, 'zx':zoom, 'zy':zoom})
            imgs.append(img2)

        sys.stdout.write("\rImage {}/{} is being processed".format(i+1,len(filenames)))
        sys.stdout.flush()
        
    print(' ... Shuffle data ...')
    imgs1 = np.asarray(imgs)
    np.random.seed(seed)
    rand = np.random.randint(imgs1.shape[0], size=imgs1.shape[0])
    return imgs1[rand, :, :, :]

tr_imgs = data_augmentation("../Datasets/training/images/")      #100 images --> 2400 images
gt_imgs = data_augmentation("../Datasets/training/groundtruth/") #100 images --> 2400 images

Image 100/100 is being processedShuffle data
Image 100/100 is being processedShuffle data


In [27]:
# create model
model = Sequential()

# add model layers
model.add(Conv2D(16, kernel_size=IMG_PATCH_SIZE, activation='relu',
                      input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

# compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
#Assign a label to a patch v
def value_to_class(v):
    df = np.sum(v)
    if df > FOREGROUND_THRESHOLD:
        return [0, 1]
    else:
        return [1, 0]

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches


# Extract label images
def extract_labels(gt_imgs):
    """Extract the labels into a 1-hot matrix [image index, label index]."""            
    imgs = gt_imgs[:TRAINING_SIZE, :, :, :]

    num_images = len(imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

    
def create_patches(im):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0, imgheight, IMG_PATCH_SIZE):
        for j in range(0, imgwidth, IMG_PATCH_SIZE):
            im_patch = im[j:j+IMG_PATCH_SIZE, i:i+IMG_PATCH_SIZE, :]
            list_patches.append(im_patch)
    return list_patches


def extract_data(tr_imgs):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """#todo
    
    imgs = tr_imgs[:TRAINING_SIZE, :, :, :]

    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [create_patches(imgs[i]) for i in range(TRAINING_SIZE)] #list of images (=list windows (=list pixels))
    
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))] # j in range number of patch per image

    return np.asarray(data)


def load_data(tr_imgs, gt_imgs):
        # Extract it into numpy arrays.
        train_data = extract_data(tr_imgs) #shape: ((400/16)^2 * 100 = 62500, 16, 16, 3)
        train_labels = extract_labels(gt_imgs)
        return train_data, train_labels

In [31]:
train_data, train_labels = load_data(tr_imgs, gt_imgs)

In [32]:
train_data[:2]

array([[[[0.4664344 , 0.4538443 , 0.40348393],
         [0.4553619 , 0.44548875, 0.41545388],
         [0.5829659 , 0.57718354, 0.56541884],
         ...,
         [0.51327723, 0.50935566, 0.4952679 ],
         [0.36658093, 0.36074167, 0.35272613],
         [0.20534872, 0.1989317 , 0.19750558]],

        [[0.6672187 , 0.65306   , 0.5964251 ],
         [0.3769305 , 0.36548874, 0.3213362 ],
         [0.5127776 , 0.5040528 , 0.48435277],
         ...,
         [0.46621838, 0.4622968 , 0.4513463 ],
         [0.50618875, 0.50191814, 0.4876281 ],
         [0.26438087, 0.2573933 , 0.25397104]],

        [[0.85806227, 0.8421706 , 0.77971303],
         [0.5202344 , 0.50722396, 0.45518234],
         [0.43434626, 0.4240528 , 0.39023513],
         ...,
         [0.4198021 , 0.41588053, 0.4080374 ],
         [0.5006678 , 0.49674627, 0.48349914],
         [0.40398875, 0.3985698 , 0.388873  ]],

        ...,

        [[0.09011009, 0.0949392 , 0.10026509],
         [0.17014714, 0.16682543, 0.16641052]

In [7]:
def train(epochs=100, validation_split=0.1):

    # Step 0: Shuffle samples
    np.random.seed(0)
    np.random.shuffle(train_data)
    # resetting the seed allows for an identical shuffling between y and x
    np.random.seed(0)
    np.random.shuffle(train_labels)

    # Step 1: Split into validation and training set     
    split_index = int(len(train_data) * (1 - validation_split))
    train_data_split = train_data[0:split_index]
    validation_data_split = train_data[split_index:len(train_data)]
    train_label_split = train_labels[0:split_index]
    validation_label_split = train_labels[split_index:len(train_data)]

    # Step 2: Give weights to classes
    c_weight = {1: 3., 
                0: 1.}

    # Step 3: Greate Generators
    train_datagen = ImageDataGenerator(
        #rotation_range=180,
        horizontal_flip=True,
        vertical_flip=True)

    validation_datagen = ImageDataGenerator()
        #rotation_range=180,
        #horizontal_flip=True,
        #vertical_flip=True)

    train_generator = train_datagen.flow(train_data_split, train_label_split, batch_size=32)
    validation_generator = validation_datagen.flow(validation_data_split, validation_label_split, batch_size=32)


    # Step 4: Early stop
    early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, 
                                        mode='max', restore_best_weights=True)

    # Finally, train the model !
    # Training
    model.fit_generator(train_generator,
                validation_data=validation_generator,
                steps_per_epoch=len(train_data_split)/32,
                epochs=epochs,
                callbacks = [early_stop_callback],
                class_weight=c_weight,
                validation_steps=len(validation_data_split)/16)

# Train model
train(epochs=30, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30





1563/1562 [==============================] - 25s 16ms/step - loss: 1.0490 - acc: 0.3430 - val_loss: 0.7094 - val_acc: 0.3003
Epoch 2/30
1563/1562 [==============================] - 17s 11ms/step - loss: 1.0374 - acc: 0.3644 - val_loss: 0.6645 - val_acc: 0.4487
Epoch 3/30
1563/1562 [==============================] - 17s 11ms/step - loss: 1.0109 - acc: 0.4377 - val_loss: 0.6700 - val_acc: 0.4831
Epoch 4/30
1563/1562 [==============================] - 18s 11ms/step - loss: 0.9743 - acc: 0.4984 - val_loss: 0.6338 - val_acc: 0.5308
Epoch 5/30
1563/1562 [==============================] - 21s 14ms/step - loss: 0.9571 - acc: 0.5177 - val_loss: 0.6320 - val_acc: 0.5401
Epoch 6/30
1563/1562 [==============================] - 22s 14ms/step - loss: 0.9548 - acc: 0.5175 - val_loss: 0.6116 - val_acc: 0.5723
Epoch 7/30
1563/1562 [==============================] - 15s 10ms/step - loss: 0.95